In [ ]:
import os
from pathlib import Path
from mne.io import read_raw, read_raw_ant
from mne_bids import BIDSPath, write_raw_bids
from tqdm.contrib import tzip
from tqdm import tqdm
from mne import set_log_level
set_log_level("Error")